<a href="https://colab.research.google.com/github/hamednasr/DDoS-Attacks/blob/main/Adaboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing modules

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sklearn
import sklearn.ensemble   
import sklearn.datasets   
import sklearn.metrics     
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving KDDTrain.csv to KDDTrain.csv


## Reading the Data

In [ ]:
data = pd.read_csv('KDDTrain.csv')

data

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Label
0,0,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
2,0,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
3,0,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,121,19,0.0,0.0,1.0,1.0,0.16,0.06,0.00,255,19,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,184,25,1.0,1.0,0.0,0.0,0.14,0.06,0.00,255,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune
125968,8,105,145,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal
125969,0,2231,384,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal
125970,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,144,8,1.0,1.0,0.0,0.0,0.06,0.05,0.00,255,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune


In [ ]:
rows, columns = np.where(pd.isnull(data))
data_filtered = data.drop(labels=rows)

print("There are %d rows having null values" %(len(data) - len(data_filtered)))

data_filtered

There are 0 rows having null values


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Label
0,0,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
2,0,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
3,0,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,121,19,0.0,0.0,1.0,1.0,0.16,0.06,0.00,255,19,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,184,25,1.0,1.0,0.0,0.0,0.14,0.06,0.00,255,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune
125968,8,105,145,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal
125969,0,2231,384,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal
125970,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,144,8,1.0,1.0,0.0,0.0,0.06,0.05,0.00,255,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune


In [ ]:

data_unduplicated = data_filtered.drop_duplicates()
print("There are %d duplicated rows" %(len(data_filtered) - len(data_unduplicated)))
data_unduplicated

There are 6356 duplicated rows


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Label
0,0,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
2,0,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
3,0,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,121,19,0.0,0.0,1.0,1.0,0.16,0.06,0.00,255,19,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,184,25,1.0,1.0,0.0,0.0,0.14,0.06,0.00,255,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune
125968,8,105,145,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal
125969,0,2231,384,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal
125970,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,144,8,1.0,1.0,0.0,0.0,0.06,0.05,0.00,255,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune


In [ ]:

X, y = data_unduplicated.iloc[:, :-1], data_unduplicated.iloc[:, -1]
print("Shape of X: %s" %str(X.shape), "- Type of X: %s" %str(type(X)))
print("Shape of y: %s" %str(y.shape), "- Type of y: %s" %str(type(y)))
y

Shape of X: (119616, 38) - Type of X: <class 'pandas.core.frame.DataFrame'>
Shape of y: (119616,) - Type of y: <class 'pandas.core.series.Series'>


0          normal
1         neptune
2          normal
3          normal
4         neptune
           ...   
125967    neptune
125968     normal
125969     normal
125970    neptune
125971     normal
Name: Label, Length: 119616, dtype: object

In [ ]:

print("The number of original classes: %d" %len(y.unique()))
y.unique()

The number of original classes: 23


array(['normal', 'neptune', 'warezclient', 'ipsweep', 'portsweep',
       'teardrop', 'nmap', 'satan', 'smurf', 'pod', 'back',
       'guess_passwd', 'ftp_write', 'multihop', 'rootkit',
       'buffer_overflow', 'imap', 'warezmaster', 'phf', 'land',
       'loadmodule', 'spy', 'perl'], dtype=object)

In [ ]:
# Transform from 23 classes to 5 main classes of Internet attacks
# normal --> normal
# back, neptune, pod, smurf, teardrop --> DOS
# ipsweep, nmap, portsweep, satan --> Probing
# ftp_write, guess_passwd, imap, multihop, phf, spy, warezclient, warezmaster --> R2L
# buffer_overflow, loadmodule, perl, rootkit --> U2R

y_array = np.array(y)
y_encoded = np.zeros((y.shape))
for i in range(len(y_array)):
    if y_array[i] in ['normal']:
        y_array[i] = 'normal'
        y_encoded[i] = 0
    elif y_array[i] in ['back', 'land', 'neptune', 'pod', 'smurf', 'teardrop']:
        y_array[i] = 'DOS'
        y_encoded[i] = 1
    elif y_array[i] in ['ipsweep', 'nmap', 'portsweep', 'satan']:
        y_array[i] = 'probing'
        y_encoded[i] = 2
    elif y_array[i] in ['ftp_write', 'guess_passwd', 'imap', 'multihop', 'phf', 'spy', 'warezclient', 'warezmaster']:
        y_array[i] = 'R2L'
        y_encoded[i] = 3
    elif y_array[i] in ['buffer_overflow', 'loadmodule', 'perl', 'rootkit']:
        y_array[i] = 'U2R'
        y_encoded[i] = 4
y_df = pd.Series(y_array)
y_df.unique()

array(['normal', 'DOS', 'R2L', 'probing', 'U2R'], dtype=object)

In [ ]:

for name in y_df.unique():
    print("The number of samples of class %s: %d" %(str(name), len(X[y_array==name])))
y_encoded[:10]

The number of samples of class normal: 67292
The number of samples of class DOS: 39893
The number of samples of class R2L: 995
The number of samples of class probing: 11384
The number of samples of class U2R: 52


array([0., 1., 0., 0., 1., 1., 1., 1., 1., 1.])

In [ ]:

data_new = X
data_new['Label'] = y_encoded
data_sorted = data_new.sort_values(by='Label')

data_sorted

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Label
0,0,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.0,0.0,0.00,0.0,0.0
71814,0,241,4198,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,0.0,0.0,1.00,0.00,1.00,2,255,1.00,0.00,0.50,0.04,0.0,0.0,0.00,0.0,0.0
71813,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.00,0.00,0.00,12,93,1.00,0.00,0.08,0.34,0.0,0.0,1.00,1.0,0.0
71812,0,45,127,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,45,260,0.0,0.0,0.0,0.0,1.00,0.00,0.01,103,200,0.85,0.02,0.85,0.01,0.0,0.0,0.11,0.0,0.0
71810,0,206,12090,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,9,0.0,0.0,0.0,0.0,1.00,0.00,0.33,6,255,1.00,0.00,0.17,0.04,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84225,0,0,5928,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,2,82,1.00,0.00,1.00,0.02,0.0,0.0,0.00,0.0,4.0
99759,158,1567,3095,0,0,0,3,0,1,4,1,0,0,1,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,2,2,1.00,0.00,0.50,0.00,0.0,0.0,0.00,0.0,4.0
108499,85,277,693,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,1,1,1.00,0.00,1.00,0.00,0.0,0.0,0.00,0.0,4.0
82251,7,226,698,0,0,0,4,0,1,0,0,0,0,4,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,1,1,1.00,0.00,1.00,0.00,0.0,0.0,0.00,0.0,4.0


In [ ]:

# Split the training set and testing set
data_array = data_sorted.to_numpy()
X_new, y_new = data_array[:, :-1], data_array[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=0)

# Feature selection
feature_selection = SelectKBest(chi2, k=10).fit(X_train, y_train)
X_train_new = feature_selection.transform(X_train)
X_test_new = feature_selection.transform(X_test)
feature_selection.get_support()


array([ True,  True,  True, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True,  True, False,  True, False, False, False, False,
       False,  True,  True, False, False, False, False, False,  True,
       False, False])

In [ ]:
param_grid = {
    'n_estimators' : [100, 1000, 2000, 4000],
    'learning_rate' : [0.001, 0.01, 0.1, 0.2]
}
searcher = GridSearchCV(AdaBoostClassifier(), param_grid = param_grid, scoring='accuracy',n_jobs=-1).fit(X_train, y_train)
y_pred = searcher.predict(X_test)
searcher.best_estimator_

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
                   learning_rate=0.001, n_estimators=4000, random_state=None)

In [ ]:
adab = AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
                   learning_rate=0.001, n_estimators=4000, random_state=None).fit(X_train_new, y_train)

In [ ]:
print("Test Accuracy:",metrics.accuracy_score(y_test, adab.predict(X_test_new))*100, chr(37))
print("Train Accuracy:",metrics.accuracy_score(y_train, adab.predict(X_train_new))*100, chr(37))

Test Accuracy: 91.0090285905367 %
Train Accuracy: 91.27304267859382 %


In [ ]:
y_tst_pred = adab.predict(X_test_new)
print(confusion_matrix(y_test, y_tst_pred))
print("==================================")
print(classification_report(y_test, y_tst_pred))

[[13156     3   282     0     0]
 [  858  6694   448     0     0]
 [  234   104  1923     0     0]
 [  214     0     0     0     0]
 [    7     0     1     0     0]]
              precision    recall  f1-score   support

         0.0       0.91      0.98      0.94     13441
         1.0       0.98      0.84      0.90      8000
         2.0       0.72      0.85      0.78      2261
         3.0       0.00      0.00      0.00       214
         4.0       0.00      0.00      0.00         8

    accuracy                           0.91     23924
   macro avg       0.52      0.53      0.53     23924
weighted avg       0.91      0.91      0.91     23924



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
std_scaler.fit(X_train_new)
X_train_scaled = std_scaler.fit_transform(X_train_new)
X_test_scaled = std_scaler.fit_transform(X_test_new)
X_train_scaled.shape

(95692, 10)

In [ ]:
param_grid = {
    'n_estimators' : [100, 1000, 2000, 4000],
    'learning_rate' : [0.001, 0.01, 0.1, 0.2]
}
searcher = GridSearchCV(AdaBoostClassifier(), param_grid = param_grid, scoring='accuracy',n_jobs=-1).fit(X_train_scaled, y_train)
y_pred = searcher.predict(X_test_scaled)
searcher.best_estimator_

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
                   learning_rate=0.001, n_estimators=2000, random_state=None)

In [ ]:
ada = AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
                   learning_rate=0.001, n_estimators=2000, random_state=None).fit(X_train_scaled,y_train)

In [ ]:
print("Test Accuracy:",ada.score(X_test_scaled,y_test)*100, chr(37))
print("Train Accuracy:",ada.score(X_train_scaled,y_train)*100, chr(37))

Test Accuracy: 54.91556595886975 %
Train Accuracy: 90.28758934916189 %


In [ ]:
y_tst_pred = ada.predict(X_test_scaled)

print(confusion_matrix(y_test, y_tst_pred))
print("==================================")
print(classification_report(y_test, y_tst_pred))

[[3935 6264 3242    0    0]
 [ 186 7760   54    0    0]
 [  65  753 1443    0    0]
 [  19    2  193    0    0]
 [   7    0    1    0    0]]
              precision    recall  f1-score   support

         0.0       0.93      0.29      0.45     13441
         1.0       0.53      0.97      0.68      8000
         2.0       0.29      0.64      0.40      2261
         3.0       0.00      0.00      0.00       214
         4.0       0.00      0.00      0.00         8

    accuracy                           0.55     23924
   macro avg       0.35      0.38      0.31     23924
weighted avg       0.73      0.55      0.52     23924



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
